###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [1]:
import pandas as pd

###Archivo kk69-w2jj.csv

Contiene información incautación de estupefacientes desde 01 de enero del año 2010 al 30 de abril del año 2024.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/kk69-w2jj.csv?$limit=2000000")
df.head()

<ipython-input-2-439c10417958>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/kk69-w2jj.csv?$limit=2000000")


,departamento,municipio,codigo_dane,clase_bien,fecha_hecho,cantidad
0,AMAZONAS,LETICIA (CT),91001000,COCAINA,1/01/2010,8.825
1,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16
2,ANTIOQUIA,ANDES,5034000,MARIHUANA,1/01/2010,10
3,ANTIOQUIA,ANGELÓPOLIS,5036000,MARIHUANA,1/01/2010,80
4,ANTIOQUIA,ARBOLETES,5051000,MARIHUANA,1/01/2010,500


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1929652 entries, 0 to 1929651
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   departamento  object
 1   municipio     object
 2   codigo_dane   object
 3   clase_bien    object
 4   fecha_hecho   object
 5   cantidad      object
dtypes: object(6)
memory usage: 88.3+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'clase_bien','cantidad']
drug_seizure = df[relevant_cols]
drug_seizure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1929652 entries, 0 to 1929651
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   codigo_dane   object
 1   departamento  object
 2   municipio     object
 3   fecha_hecho   object
 4   clase_bien    object
 5   cantidad      object
dtypes: object(6)
memory usage: 88.3+ MB


### Convertir columna fecha_hecho a tipo date

In [5]:
# Hacer una copia explícita del DataFrame
drug_seizure_ = drug_seizure.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
drug_seizure_['fecha_hecho'] = pd.to_datetime(drug_seizure_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
drug_seizure_.loc[:, 'fecha_hecho'] = drug_seizure_['fecha_hecho'].dt.year

### Verificar valores nulos

In [6]:
drug_seizure_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
clase_bien,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [7]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = [ 'clase_bien', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(drug_seizure_[column].unique())
    print()


Categorías en la columna 'clase_bien':
['COCAINA' 'MARIHUANA' 'BASUCO' 'BASE DE COCA' 'HEROINA']

Categorías en la columna 'departamento':
['AMAZONAS' 'ANTIOQUIA' 'ATLÁNTICO' 'BOLÍVAR' 'BOYACÁ' 'CAQUETÁ' 'CAUCA'
 'CESAR' 'CUNDINAMARCA' 'GUAINÍA' 'HUILA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'QUINDÍO' 'RISARALDA' 'SANTANDER' 'VALLE' 'CALDAS'
 'CHOCÓ' 'CÓRDOBA' 'GUAVIARE' 'TOLIMA' 'PUTUMAYO' 'SUCRE' 'SAN ANDRÉS'
 'CASANARE' 'GUAJIRA' 'ARAUCA' 'VICHADA' 'VAUPÉS' 'NEW YORK' 'FLORIDA'
 'QUITO' 'NEW JERSEY' 'MADRID' 'KINGSTON' 'CALIFORNIA']

Categorías en la columna 'municipio':
['LETICIA (CT)' 'AMALFI' 'ANDES' ... 'Guapotá' 'Jesús María'
 'Santa Rosalía']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [8]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [9]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    drug_seizure_[col] = drug_seizure_[col].apply(remove_accents_and_special_chars)

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [10]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df drug_seizure_

In [11]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
drug_seizure_['codigo_dane'] = drug_seizure_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = drug_seizure_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [12]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [8 7 1 5]


In [13]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_dígitos_codigo_dane,count,percentage
0,8,1313889,0.680894
1,7,580828,0.301001
2,5,34893,0.018083
3,1,42,0.000022


In [14]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = drug_seizure_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
   codigo_dane
0     91001000
28    13001000
29    13001000
30    13001000
31    15600000

Muestra de registros con longitud 7:
  codigo_dane
1     5031000
2     5034000
3     5036000
4     5051000
5     5088000

Muestra de registros con longitud 5:
        codigo_dane
1894759       44942
1894760       44962
1894761       44974
1894762       44986
1894763       45014

Muestra de registros con longitud 1:
      codigo_dane
17480           -
17481           -
18421           0
25087           -
27374           -



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 68.1% tiene 8 dígitos (al parecer se le adicionaron 3 ceeros al final del código que realmente es de 5 dígitos), el 30.1% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió), el 1.8% tienen un código de 5 dígitos que pueden estar correctos ya que el código real tiene ese numero de dígitos, no obstante se debe verificar. También hay un total de 42 registros con un código de longitud 1 caracter que claramente están errados.

  - Verificar si los codigo_dane de 5 dígitos corresponden a códigos reales de municipios

In [15]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    return only_in_list1


In [16]:
# Filas con código de 5 dígitos
code_5 = drug_seizure_[drug_seizure_['codigo_dane'].str.len() == 5]
len(code_5)

34893

In [17]:
# Comparar listas de códigos de los dos df
len(compare_lists(code_5['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Departamentos en drug_seizure_", "Departamentos dept_mpios_codes"))

Departamentos en drug_seizure_ que no están en Departamentos dept_mpios_codes:
{'45172', '45189', '45107', '45015', '45278', '45220', '45136', '44998', '45227', '45008', '45237', '45277', '45061', '45256', '44938', '44977', '44949', '45179', '45048', '45183', '44930', '45001', '44952', '44956', '45155', '45098', '45146', '45232', '45086', '44983', '45145', '45099', '45285', '45033', '45127', '45282', '45245', '45276', '45135', '45012', '45224', '45000', '45238', '44933', '45021', '45219', '44995', '44988', '45094', '45029', '45104', '45165', '45260', '45225', '45141', '45266', '45115', '44970', '45139', '45128', '45101', '45058', '45207', '45116', '45092', '44993', '45270', '45052', '45195', '44967', '45188', '45153', '45120', '45198', '45137', '45124', '45287', '45283', '45055', '45006', '45016', '45159', '45068', '45046', '45223', '45156', '45231', '45180', '45230', '44931', '45121', '45259', '44976', '45014', '45075', '45248', '45106', '45109', '45144', '45096', '44996', '45157', '4

365

Nota: De los 34.893 códigos de 5 dígitos, 365 no corresponden a códigos reales. Más adelante abordamos este asunto

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [18]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = drug_seizure_[drug_seizure_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [19]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
drug_seizure_['codigo_dane'] = drug_seizure_['codigo_dane'].apply(add_zero_if_length_7)

In [20]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(drug_seizure_[drug_seizure_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [21]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
drug_seizure_['codigo_dane'] = drug_seizure_['codigo_dane'].apply(trim_last_3_if_length_8)

In [22]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(drug_seizure_[drug_seizure_['codigo_dane'].str.len() == 8])

0

  - Analizar los códigos que tienen 1 dígito

In [23]:
# Filas con código de 1 dígitos
code_1 = drug_seizure_[drug_seizure_['codigo_dane'].str.len() == 1]
print(f'No. Registros con con código de 1 dígito {len(code_1)}')
print(f'valores únicos de "codigo_dane" en las fila con codigo de longitud =1: {code_1["codigo_dane"].unique()}')
print(f'valores únicos de "departamento" en las fila con codigo de longitud =1: {code_1["departamento"].unique()}')

No. Registros con con código de 1 dígito 42
valores únicos de "codigo_dane" en las fila con codigo de longitud =1: ['-' '0']
valores únicos de "departamento" en las fila con codigo de longitud =1: ['NEW YORK' 'FLORIDA' 'QUITO' 'NEW JERSEY' 'MADRID' 'KINGSTON'
 'CALIFORNIA' 'NARINO']


Nota: Corresponden a incautaciones hechas en el exterior por ello no tiene codigo del municipio, salvo la de NARINO, pero los dos registros sin codigo_dane de este departamento no tienen el municipio. Por lo tanto se decide eliminar estas filas

In [24]:
code_1[code_1['departamento'] == 'NARINO']

,codigo_dane,departamento,municipio,fecha_hecho,clase_bien,cantidad
748798,-,NARINO,-,2018,COCAINA,36.650
1726042,-,NARINO,-,2018,COCAINA,36.650


In [25]:
# Eliminar registros con codigo_dane de un caracter
drug_seizure_ = drug_seizure_[~(drug_seizure_['codigo_dane'].str.len() == 1)]

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

  - Comparar nombres de departamentos en los dataframes

In [26]:
# Comparar listas de departamento
compare_lists(drug_seizure_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en drug_seizure_", "Departamentos dept_mpios_codes")

Departamentos en drug_seizure_ que no están en Departamentos dept_mpios_codes:
{'VALLE', 'GUAJIRA', 'SAN ANDRES'}


{'GUAJIRA', 'SAN ANDRES', 'VALLE'}

  - Solucionar discrepancias en nombres de departamento

In [27]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe drug_seizure_
drug_seizure_['departamento'] = drug_seizure_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [28]:
# Comparar listas de departamento
compare_lists(drug_seizure_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en drug_seizure_", "Departamentos dept_mpios_codes")

Departamentos en drug_seizure_ que no están en Departamentos dept_mpios_codes:
set()


set()

  - Verificar que todos los códigos dane tengan la misma extensión

In [29]:
drug_seizure_['codigo_dane'].apply(len).nunique()

1

In [30]:
drug_seizure_['codigo_dane'].apply(len).value_counts()

,count
codigo_dane,
5,1929610


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [31]:
# Comparar listas de codigos
result = compare_lists(drug_seizure_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
                "Códigos de municipios en drug_seizure_", "Códigos de municipios en dept_mpios_codes")
print(f'Total codigos de municipios errados: {len(result)} ')

Códigos de municipios en drug_seizure_ que no están en Códigos de municipios en dept_mpios_codes:
{'45172', '45189', '45107', '45015', '45220', '45278', '45136', '44998', '45227', '45008', '45237', '45277', '45061', '45256', '44938', '44977', '44949', '45179', '45048', '45183', '44930', '45001', '44952', '44956', '45155', '45098', '45146', '45232', '45086', '44983', '45145', '45099', '45285', '45033', '45127', '45282', '45245', '45135', '45276', '45012', '45224', '45000', '45238', '44933', '45021', '45219', '44995', '44988', '45094', '45029', '45104', '45165', '45260', '45225', '45141', '45266', '45115', '44970', '45139', '45128', '45101', '45058', '45207', '45116', '45092', '44993', '45270', '45052', '45195', '44967', '45188', '45153', '45120', '45198', '45137', '45124', '45287', '45283', '45055', '45006', '45016', '45159', '45068', '45046', '45223', '45231', '45156', '45180', '45121', '44931', '45230', '45259', '44976', '45014', '45248', '45109', '45106', '45075', '45144', '45096', '

  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [32]:
drug_seizure_[list[result]]

,codigo_dane,departamento,municipio,fecha_hecho,clase_bien,cantidad
0,91001,AMAZONAS,LETICIA (CT),2010,COCAINA,8.825
1,05031,ANTIOQUIA,AMALFI,2010,MARIHUANA,16
2,05034,ANTIOQUIA,ANDES,2010,MARIHUANA,10
3,05036,ANTIOQUIA,ANGELOPOLIS,2010,MARIHUANA,80
4,05051,ANTIOQUIA,ARBOLETES,2010,MARIHUANA,500
...,...,...,...,...,...,...
1929647,45055,VICHADA,PUERTO CARRENO (CT),2023,MARIHUANA,35
1929648,45065,VICHADA,PUERTO CARRENO (CT),2023,MARIHUANA,20
1929649,45079,VICHADA,PUERTO CARRENO (CT),2023,MARIHUANA,2
1929650,45183,VICHADA,PUERTO CARRENO (CT),2023,MARIHUANA,22.950


Sabemos:

- El df dept_mpio_codes en la columna dept_mpio_code tiene los códigos reales de los municipios de Colombia (Obtenidos del DANE)
- El df drug_seizure tiene 365 códigos de municipios que están errados
- Los dos df tienen una columna con el nombre del departamento perfectamente ajustada, también tienen una columna con el nombre del municipio pero ésta no está ajusta (algunos nombres de municipios pueden estar escritos en forma diferente).

Enfoque a aplicar:

Creamos un nueva columna combinada con nombre de departemento y municipio en ambos df, y luego con coincidencia difusa comparamos las dos cadenas de texto para medir su similitud y con este resultado mapeamos los códigos reales de los municipios para que se corrija el error.

 - Instalar la thefuzz en el entorno de google colab, para realizar coincidencia difusa

In [33]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.1 MB/s eta 0:00:00


  - Aplicar thefuzz para coincidencia difusa

In [34]:
# Crear una clave única de departamento + municipio en ambos datasets
drug_seizure_['dept_mpio'] = drug_seizure_['departamento'] + '_' + drug_seizure_['municipio']
dept_mpios_codes['dept_mpio'] = dept_mpios_codes['dept_name'] + '_' + dept_mpios_codes['mupio_name']

In [35]:
# Crear un diccionario de municipios y códigos basado en dept_mpios_codes
municipios_dict = dict(zip(dept_mpios_codes['dept_mpio'], dept_mpios_codes['dept_mpio_code']))

In [36]:
from thefuzz import process

# Funcion para Usar fuzzy matching (thefuzz)
def get_best_match(row, municipios_dict, threshold=80):
    dept_mpio_sexual = row['departamento'] + '_' + row['municipio']

    # Buscar la mejor coincidencia en dept_mpios_codes usando fuzzy matching
    best_match, score = process.extractOne(dept_mpio_sexual, municipios_dict.keys())

    # Si la similitud supera el umbral definido, devolver el código mapeado
    if score >= threshold:
        return municipios_dict[best_match]
    else:
        return row['codigo_dane']  # Mantener el código original si no hay coincidencia segura


In [37]:
# Aplicar la función solo a las filas donde sabemos que el 'codigo_dane' esta errado, es decir los registros que estan en el conjunto: result anteriormente encontrado

drug_seizure_['codigo_dane'] = drug_seizure_.apply(
    lambda row: get_best_match(row, municipios_dict, threshold=80) if row['codigo_dane'] in result else row['codigo_dane'],
    axis=1
)

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [38]:
# Comparar listas de códigos
result = compare_lists(drug_seizure_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en drug_seizure_", "Códigos de municipios en dept_mpios_codes")
print(f'Total códigos de municipios errados: {len(result)} ')

Códigos de municipios en drug_seizure_ que no están en Códigos de municipios en dept_mpios_codes:
set()
Total códigos de municipios errados: 0 


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [39]:
# Eliminar columnas innecesarias
columns_to_drop = [ 'departamento', 'municipio', 'dept_mpio']
final_drug_seizure = drug_seizure_.drop(columns=columns_to_drop)

In [40]:
# Adicionar columna para trazabilidad de la fuente
final_drug_seizure['source_id'] = 14

In [41]:
final_drug_seizure.columns

Index(['codigo_dane', 'fecha_hecho', 'clase_bien', 'cantidad', 'source_id'], dtype='object')

In [42]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'clase_bien': 'type_of_drug',
    'codigo_dane': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_drug_seizure.rename(columns=translation_map, inplace=True)

In [45]:
#Estructura final del dataset a integrar a la base de datos
final_drug_seizure.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1929610 entries, 0 to 1929651
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   dane_code         object
 1   year_of_incident  int32 
 2   type_of_drug      object
 3   amount            object
 4   source_id         int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 81.0+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_drug_seizure.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/drug_seizure.csv', index=False)